# Let's start with Multimodal Deep Archetypal Analysis 

### Archetypal Analysis

### Can we learn it as a map?

### A joint optmization view of the problem

### Initialize A and B

### Finding the best number of archetypes

### What about non-tabular data?